In [ ]:
import PyPDF2
import pandas as pd
from rapidfuzz import fuzz

# ... (same code as before to extract PDF paragraphs and prepare Excel data) ...

matching_results = []

for paragraph_number, paragraph_text in enumerate(pdf_paragraphs, start=1):
    matched_data = []
    
    for index, row in excel_data.iterrows():
        title = row['Title']
        description = row['Description']
        objective = row['Objective']
        
        title_score = fuzz.ratio(title.lower(), paragraph_text.lower())
        description_score = fuzz.ratio(description.lower(), paragraph_text.lower())
        objective_score = fuzz.ratio(objective.lower(), paragraph_text.lower())
        
        if title_score >= 80:
            matched_data.append({
                'Matched Title': title,
                'Matched Description': description,
                'Matched Objective': objective,
                'Matched Score': title_score
            })
        
        if description_score >= 80:
            matched_data.append({
                'Matched Title': title,
                'Matched Description': description,
                'Matched Objective': objective,
                'Matched Score': description_score
            })
        
        if objective_score >= 80:
            matched_data.append({
                'Matched Title': title,
                'Matched Description': description,
                'Matched Objective': objective,
                'Matched Score': objective_score
            })

    matching_results.append({
        'Paragraph': paragraph_text,
        'Matched Data': matched_data,
        'Page Number': paragraph_number
    })

# Create a new DataFrame to store the results
result_data = []
for result in matching_results:
    paragraph = result['Paragraph']
    page_number = result['Page Number']
    for matched_item in result['Matched Data']:
        matched_title = matched_item['Matched Title']
        matched_description = matched_item['Matched Description']
        matched_objective = matched_item['Matched Objective']
        matched_score = matched_item['Matched Score']
        result_data.append({
            'Paragraph': paragraph,
            'Page Number': page_number,
            'Matched Title': matched_title,
            'Matched Description': matched_description,
            'Matched Objective': matched_objective,
            'Matched Score': matched_score
        })

matching_df = pd.DataFrame(result_data)

# Save the matching results to a CSV file
#matching_df.to_csv('matching_results.csv', index=False)

In [ ]:
import PyPDF2
import pandas as pd
from rapidfuzz import process

# ... (same code as before to extract PDF paragraphs and prepare Excel data) ...

matching_results = []

for paragraph_number, paragraph_text in enumerate(pdf_paragraphs, start=1):
    for index, row in excel_data.iterrows():
        title = row['Title']
        description = row['Description']
        objective = row['Objective']
        objective_name = row['Objective Name']  # Added Objective Name
        
        matched_title_keywords = []
        matched_description_keywords = []
        matched_objective_keywords = []
        matched_objective_name = None  # Initialize matched Objective Name
        
        combined_score = 0  # Initialize combined score
        
        for keyword in title.split('\n'):
            keyword = keyword.strip()
            
            best_match_info = process.extractOne(keyword.lower(), paragraph_text.lower().split())
            best_match = best_match_info[0]
            match_score = best_match_info[1]
            
            if match_score >= 80:
                matched_title_keywords.append(f"{best_match} (Score: {match_score})")
                combined_score += match_score  # Add to combined score
        
        for keyword in description.split('\n'):
            keyword = keyword.strip()
            
            best_match_info = process.extractOne(keyword.lower(), paragraph_text.lower().split())
            best_match = best_match_info[0]
            match_score = best_match_info[1]
            
            if match_score >= 80:
                matched_description_keywords.append(f"{best_match} (Score: {match_score})")
                combined_score += match_score  # Add to combined score
        
        for keyword in objective.split('\n'):
            keyword = keyword.strip()
            
            best_match_info = process.extractOne(keyword.lower(), paragraph_text.lower().split())
            best_match = best_match_info[0]
            match_score = best_match_info[1]
            
            if match_score >= 80:
                matched_objective_keywords.append(f"{best_match} (Score: {match_score})")
                combined_score += match_score  # Add to combined score
        
        # Matching Objective Name with the paragraph text
        objective_name_score = process.extractOne(objective_name.lower(), paragraph_text.lower().split())
        if objective_name_score[1] >= 80:
            matched_objective_name = f"{objective_name_score[0]} (Score: {objective_name_score[1]})"
        
        if (
            matched_title_keywords or
            matched_description_keywords or
            matched_objective_keywords or
            matched_objective_name  # Checking if Objective Name is matched
        ):
            matching_results.append({
                'Paragraph': paragraph_text,
                'Matched Title Keywords': ', '.join(matched_title_keywords),
                'Matched Description Keywords': ', '.join(matched_description_keywords),
                'Matched Objective Keywords': ', '.join(matched_objective_keywords),
                'Matched Objective Name': matched_objective_name,  # Adding matched Objective Name
                'Page Number': paragraph_number,
                'Original Title': title,
                'Original Description': description,
                'Original Objective': objective,
                'Objective Name': objective_name,
                'Combined Match Score': combined_score  # Adding combined score
            })

# Create a new DataFrame to store the results
matching_df = pd.DataFrame(matching_results)

# Save the matching results to a CSV file
matching_df.to_csv('matching_results.csv', index=False)
